In [1]:
import findspark
findspark.init()

import pyspark
import random

sc = pyspark.SparkContext(appName="MNIST")
sc.version

'2.2.0.2.6.3.0-235'

In [2]:
import os
import glob
import numpy as np

#dataFolder='/user/gratienj/Data/MNIST'
dataFolder='/user/gratienj'
filename=os.path.join(dataFolder,'train.csv')

In [3]:
# Transformation du fichier texte en RDD de valeurs
## Données d'apprentissage
trainMnist = sc.textFile(filename).map(lambda l: [float(x) for x in l.split(',')])
trainMnist.count() # taille de l'échantillon
#test

60000

In [4]:
# Importation des packages
import time
from numpy import array
from pyspark.mllib.regression import LabeledPoint

In [5]:
# Mise au format "labeled point" spécifique de MLlib "(label (v1, v2...vp])"
trainMnistLP=trainMnist.map(lambda line: LabeledPoint(line[0],[line[1:754]]))
trainMnistLP.count()

60000

In [6]:
## Même chose pour les données de test
test_filename=os.path.join(dataFolder,'test.csv')
testMnist = sc.textFile(test_filename).map(lambda l: [float(x) for x in l.split(',')])
testMnist.count() # taille de l'échantillon

10000

In [7]:
testData=testMnist.map(lambda line: LabeledPoint(line[0],[line[1:754]]))
testData.take(10)

[LabeledPoint(7.0, [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,84.0,185.0,159.0,151.0,60.0,36.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,222.0,254.0,254.0,254.0,254.0,241.0,198.0,198.0,198

In [8]:
tauxEch=0.1 # tester pour des tailles croissantes d'échantillon d'apprentissage
(trainData, DropDatal) = trainMnistLP.randomSplit([tauxEch, 1-tauxEch])
trainData.count()

5925

In [9]:
## Logistic Regression
import pyspark.mllib.regression
from pyspark.mllib.classification import LogisticRegressionWithLBFGS
# Les paramètres ont leurs valeurs par défaut
time_start=time.time()
model_lrm = LogisticRegressionWithLBFGS.train(trainData, 
            iterations=100, initialWeights=None, regParam=0.01, 
            regType='l2', intercept=False, corrections=10, tolerance=0.0001, 
            validateData=True, numClasses=10)
#Parameters:
# data – The training data, an RDD of LabeledPoint.
# iterations – The number of iterations (default: 100).
# initialWeights – The initial weights (default: None).
# regParam – The regularizer parameter (default: 0.01).
# regType – The type of regularizer used for training our model.“l1” for using L1 regularization
#           “l2” for using L2 regularization None for no regularization(default: “l2”)
# intercept – Boolean parameter which indicates the use or not of the augmented representation for training data 
#             (i.e. whether bias features are activated or not, default: False).
# corrections – The number of corrections used in the LBFGS update (default: 10).
# tolerance – The convergence tolerance of iterations for L-BFGS (default: 1e-4).
# validateData – Boolean parameter which indicates if the algorithm should validate data before training.(default:True)
# numClasses – The number of classes (i.e., outcomes) a label can take in Multinomial Logistic Regression (default: 2).
 
time_end=time.time()
time_lrm=(time_end - time_start)
print("LR prend %d s" %(time_lrm)) # (104s avec taux=1)

LR prend 20 s


In [10]:
predictions = model_lrm.predict(testData.map(lambda x: x.features))
labelsAndPredictions = testData.map(lambda lp: lp.label).zip(predictions)
testErr = labelsAndPredictions.filter(lambda line: line[0] != line[1]).count() / float(testData.count())
print('Test Error = ' + str(testErr)) # (0.08 avec taux =1)

Test Error = 0.0932


In [11]:
from pyspark.mllib.tree import RandomForest
time_start=time.time()
model_rf = RandomForest.trainClassifier(trainData, numClasses = 10,
        categoricalFeaturesInfo = {}, numTrees = 100, 
        featureSubsetStrategy='auto', impurity='gini', maxDepth=12,
        maxBins=32, seed=None)
#Parameters:
#•data – Training dataset: RDD of LabeledPoint. Labels should take values {0, 1, ..., numClasses-1}.
#•numClasses – number of classes for classification.
#•categoricalFeaturesInfo – Map storing arity of categorical features. E.g., an entry (n -> k) indicates that feature 
# n is categorical with k categories indexed from 0: {0, 1, ..., k-1}.
#•numTrees – Number of trees in the random forest.
#•featureSubsetStrategy – Number of features to consider for splits at each node. Supported: “auto” (default), “all”, 
# “sqrt”, “log2”, “onethird”. If “auto” is set, this parameter is set based on numTrees: if numTrees == 1,set to “all”; 
# if numTrees > 1 (forest) set to “sqrt”.
#•impurity – Criterion used for information gain calculation. Supported values: “gini” (recommended) or “entropy”.
#•maxDepth – Maximum depth of the tree. E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes. 
#(default: 4)
#•maxBins – maximum number of bins used for splitting features (default: 32)
#•seed – Random seed for bootstrapping and choosing feature subsets.
 
model_rf.numTrees()
model_rf.totalNumNodes()
time_end=time.time()
time_rf=(time_end - time_start)
print("RF takes %d s" %(time_rf))#

RF takes 91 s


In [12]:
predictions = model_rf.predict(testData.map(lambda x: x.features))
labelsAndPredictions = testData.map(lambda lp: lp.label).zip(predictions)
testErr = labelsAndPredictions.filter(lambda line : line[0] != line[1]).count() / float(testData.count())
print('Test Error = ' + str(testErr)) # (0.08 avec taux =1)

Test Error = 0.0602
